In [2]:
import numpy as np
import networkx as nx
from itertools import product
import random
import matplotlib.pyplot as plt

In [109]:
partition_table5a = [
    ["UUU", "UUC", "UUA"],
    ["UUG", "CUG", "AUG"],
    ["CUU", "CUC", "CUA"],
    ["AUU", "AUC", "AUA"],
    ["GUU", "GUC", "GUA"],
    ["GUG", "GCG", "GAG", "GGG"],
    ["UCU", "UCC", "UCA"],
    ["UCG", "CCG", "ACG"],
    ["CCU", "CCC", "CCA"],
    ["ACU", "ACC", "ACA"],
    ["GCU", "GCC", "GCA"],
    ["UAU", "UAC", "UAA"],
    ["UAG", "CAG", "AAG"],
    ["CAU", "CAC", "CAA"],
    ["AAU", "AAC", "AAA"],
    ["GAU", "GAC", "GAA"],
    ["UGU", "UGC", "UGA"],
    ["UGG", "CGG", "AGG"],
    ["CGU", "CGC", "CGA"],
    ["AGU", "AGC", "AGA"],
    ["GGU", "GGC", "GGA"],
]

In [110]:
print(len(set([item for sublist in partition_table5a for item in sublist])))

64


In [111]:
length = 3
alphabet = ['A', 'C', 'G', 'U']
nodes = [''.join(p) for p in product(alphabet, repeat=length)]

In [112]:
swap_index =  {
    0: {#U
        0: 
        #C    #A    #G   
        {1: 0, 2: 1, 3: 2},
        #C
        1:
        #A    #G
        {2: 3, 3: 4},
        #A
        2:
        #G
        {3: 5}
        },
    1: {#A
        0: 
        #C    #A    #G   
        {1: 6, 2: 7, 3: 8},
        #C
        1:
        #A    #G
        {2: 9, 3: 10},
        #A
        2:
        #G
        {3: 11}
        },
    2: {#A
        0: 
        #C    #A    #G   
        {1: 12, 2: 13, 3: 14},
        #C
        1:
        #A    #G
        {2: 15, 3: 16},
        #A
        2:
        #G
        {3: 17}
        },
}

table_ones = np.ones(18)

table_altered = [1, 1, 1, 1, 1, 1,
                 1, 1, 1, 1, 1, 1,
                 2, 4, 2, 2, 4, 2]

In [24]:
def weights(s1, s2, table):
    if sum(a != b for a, b in zip(s1, s2)) == 1:
        if s1[0] != s2[0]:
            # base 1
            letter1 = s1[0]
            letter2 = s2[0]
            base = 0
        elif s1[1] != s2[1]:
            # base 2
            letter1 = s1[1]
            letter2 = s2[1]
            base = 1
        else:
            # base3
            letter1 = s1[2]
            letter2 = s2[2]
            base = 2
    
        l1 = ['U', 'C', 'A', 'G'].index(letter1)
        l2 = ['U', 'C', 'A', 'G'].index(letter2)
        if l1 > l2:
            l1, l2 = l2, l1
        return table[swap_index[base][l1][l2]]

In [25]:
def conductance(S, G):
    numerator = 0
    denominator = 0
    for edge, w in nx.get_edge_attributes(G, "weight").items():
        node1, node2 = edge
        if node1 in S and node2 in S:
            denominator += 2 * w
        else:
            numerator += w
            denominator += w

    return numerator / denominator

In [117]:
def mean_conductance(table, code):
    conductances = []
    for S in code:
        G = nx.Graph()
        for i, node1 in enumerate(S):
            for node2 in nodes:
                if node2 == node1:
                    continue
                w = weights(node1, node2, table)
                if w:
                    G.add_edge(node1, node2, weight=w)
                    
        conductances.append(conductance(S, G))
    return np.mean(conductances)

In [122]:
def fitness(table, code):
    return 1 - mean_conductance(table, code)

In [123]:
print(mean_conductance(table_ones, partition_table5a))
print(fitness(table_ones, partition_table5a))

0.7724867724867727
0.22751322751322733


In [158]:
def random_partition():
    partition = []
    for i in range(21):
        partition.append(i)
    for i in range(64 - 21):
        partition.append(random.randint(0, 20))
    random.shuffle(partition)
    return partition

def partition_to_codons(partition):
    partition_codons = [[] for _ in range(21)]
    for i, group in enumerate(partition):
        partition_codons[group].append(nodes[i])
    return partition_codons

In [163]:
partition = partition_to_codons(random_partition())
print(mean_conductance(table_ones, partition))

0.973721340388007


In [200]:
def initialize_population(pop_size=100):
    partitions = [random_partition() for _ in range(pop_size)]
    return partitions

In [202]:
def crossover(parent1, parent2):
    child = parent1.copy()
    for i in range(64):
        if np.random.rand() > 0.5:
            child[i] = parent2[i]
        if len(set(child)) != 21:
            child[i] = parent1[i]
    return child

p1 = random_partition()
p2 = random_partition()

print(crossover(p1, p2))

[12, 5, 4, 19, 6, 1, 14, 3, 3, 19, 14, 0, 10, 11, 3, 4, 13, 8, 20, 2, 3, 18, 18, 7, 8, 19, 15, 6, 14, 2, 8, 5, 10, 10, 10, 2, 10, 6, 16, 10, 17, 11, 12, 2, 4, 6, 6, 3, 3, 10, 0, 20, 4, 20, 11, 13, 0, 9, 1, 8, 18, 17, 15, 6]


In [227]:
def mutate(parent, mutation_rate=0.4):
    child = parent.copy()
    if np.random.rand() < mutation_rate:
        return child
    
    while True:
        i, j = np.random.choice(64, 2, replace=False)
        child[i], child[j] = parent[j], parent[i]
        if len(set(child)) == 21:
            break
        else:
            child[i], child[j] = parent[i], parent[j]
    return child
        
p = random_partition()

In [244]:
def evolutionary_algorithm(fitness, pop_size=150, crossover_rate=0.6, max_generations=1000, bias_factor=1, start=None):
    
    if start is not None:
        population = [start for _ in range(pop_size)]
    else:
        population = initialize_population(pop_size)
    
    best_fitness = 0
    generations = 0
    
    probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
    probabilities /= np.sum(probabilities)
    
    while generations < max_generations:
        fitness_values = np.array([fitness(table_ones, partition_to_codons(ind)) for ind in population])
        sorted_indices = np.argsort(fitness_values)[::-1]
        sorted_population = []
        for i in sorted_indices:
            sorted_population.append(population[i])
        population = sorted_population
        new_population = []
      
        for i in range(int(crossover_rate * pop_size)):
            parent1_index, parent2_index = np.random.choice(pop_size, size=2, p=probabilities, replace=False)
            parent1, parent2 = population[parent1_index], population[parent2_index]
            child = crossover(parent1, parent2)
            new_population.append(child)
        
        while len(new_population) < pop_size:
            new_population.append(population[np.random.choice(pop_size, p=probabilities)].copy())
        
        new_population = np.array([mutate(ind) for ind in new_population])
        
        new_best_fitness = max(fitness_values)
        if new_best_fitness > best_fitness:
            best_fitness = new_best_fitness
        print(f"{best_fitness:.5f}, {new_best_fitness:.5f}", generations)
        
        population = new_population
        generations += 1
    
    fitness_values = np.array([fitness(table_ones, partition_to_codons(ind)) for ind in population])
    sorted_indices = np.argsort(fitness_values)[::-1]
    population = population[sorted_indices]
    best_individual = population[0]
    
    return best_individual

In [245]:
best_ind = evolutionary_algorithm(fitness, pop_size=200, max_generations=100, bias_factor=2.0, start=best_ind)

0.20388, 0.20388 0
0.20388, 0.20388 1
0.21235, 0.21235 2
0.21235, 0.21235 3
0.21235, 0.21235 4
0.21235, 0.21235 5
0.21235, 0.21235 6
0.21235, 0.21235 7
0.21340, 0.21340 8
0.21340, 0.21340 9
0.21340, 0.21340 10
0.21340, 0.21235 11
0.21340, 0.21235 12
0.21340, 0.21235 13
0.21340, 0.21340 14
0.21340, 0.21235 15
0.21340, 0.21164 16
0.21340, 0.21164 17
0.21340, 0.21164 18
0.21340, 0.20811 19
0.21340, 0.21340 20
0.21340, 0.20811 21
0.21340, 0.21235 22
0.21340, 0.21235 23
0.21340, 0.21235 24
0.21340, 0.21235 25
0.21340, 0.21235 26
0.21340, 0.20811 27
0.21340, 0.20811 28
0.21340, 0.21235 29
0.21340, 0.21340 30
0.21340, 0.21235 31
0.21340, 0.21058 32
0.21340, 0.21235 33
0.21340, 0.21235 34
0.21517, 0.21517 35
0.21517, 0.20988 36
0.21517, 0.20988 37
0.21517, 0.21235 38
0.21517, 0.21235 39
0.21517, 0.21517 40
0.21517, 0.21517 41
0.21517, 0.21517 42
0.21517, 0.21235 43
0.21517, 0.21235 44
0.21517, 0.21235 45
0.21517, 0.21235 46
0.21517, 0.21235 47
0.21517, 0.21376 48
0.21517, 0.21235 49
0.21517, 0

In [246]:
print(partition_to_codons(best_ind))

[['AGG', 'CGG', 'GGG', 'UGG'], ['GAA', 'GCA', 'GUA', 'UCA'], ['AGA', 'CGA', 'GGA', 'UGA'], ['AGU', 'GGU'], ['CCC', 'GCC', 'UCC'], ['AGC', 'CGC', 'GGC', 'UGC'], ['AAA', 'AAC', 'AAG', 'AAU'], ['AUC', 'CUC', 'UUC'], ['AUG', 'CUG'], ['UAA', 'UAC', 'UAU'], ['AUU', 'CUU', 'GUU', 'UUU'], ['AUA', 'UUA'], ['UAG', 'UCG', 'UUG'], ['CCA', 'CUA'], ['GUC', 'GUG'], ['CCU', 'GCU', 'UCU'], ['CGU', 'UGU'], ['ACA', 'ACC', 'ACG', 'ACU'], ['GAC', 'GAG', 'GAU'], ['CCG', 'GCG'], ['CAA', 'CAC', 'CAG', 'CAU']]
